<a href="https://colab.research.google.com/github/sanjay-thiyagarajan/score-predictor/blob/main/CricPredictor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!wget 'https://internalapp.nptel.ac.in/contest/reference_docs/ipl_csv2.zip'
!mkdir '/content/dataset'
!unzip '/content/ipl_csv2.zip' -d '/content/dataset'
!wget "https://internalapp.nptel.ac.in/contest/reference_docs/sampleinput_files.zip"
!mkdir tests
!unzip '/content/sampleinput_files.zip' -d '/content/tests/'

In [69]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score
import warnings
warnings.filterwarnings('ignore')

def parse(power_play_df, test_df):
  test_df = test_df.drop(columns=['venue','innings','batting_team','bowling_team'])
  cols = list(power_play_df.drop(columns=['runs_off_bat']).columns)
  people = np.array(test_df['batsmen'] + ',' + test_df['bowlers'])
  people = people.flatten()[0].split(',')
  df = pd.DataFrame()
  for c in cols:
    df[c]=[0]
  for p in people:
    df[p] = 1
  return df
  
def predictRuns(df):
  power_play_df = pd.read_csv('https://raw.githubusercontent.com/sanjay-thiyagarajan/score-predictor/main/processed.csv')
  power_play_df = power_play_df.drop(columns=['Unnamed: 0', 'Unnamed: 3'])
  train_list = list(power_play_df.drop(columns=['runs_off_bat']).columns)
  test_list = list(parse(power_play_df, df).columns)
  missing = list(set(train_list) ^ set(test_list))
  for m in missing:
    power_play_df[m] = [0]*power_play_df.shape[0]
  X_train = np.array(power_play_df.drop(columns=['runs_off_bat']))
  Y_train = np.array(power_play_df[['runs_off_bat']])
  reg = RandomForestRegressor(n_estimators=50)
  reg.fit(X_train, Y_train)
  X_test = np.array(parse(power_play_df, df))
  return reg.predict(X_test)[0]

In [76]:
dd = pd.read_csv('/content/tests/sampleinput_files/Apr-29-inn2-match1.csv')
score = predictRuns(dd)

In [77]:
score

35.72